<a href="https://colab.research.google.com/github/moshimoshisama/Expedia_Analysis/blob/main/2_preliminary_analysis_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook read input datas as pyspark dataframe, in order to efficiently handle the data. At the end of this notebook, I realized that the dataset has very unbalanced classes with extremely low click through rate and transaction rate. 

# **Part I: Pyspark installation & build session**

In [ ]:
!pip install pyspark py4j findspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 66.2 MB/s 
     |████████████████████████████████| 198 kB 82.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4bf3aabfd85ea21b886f82cdeab16f12ed455d79c8f194eba55ca9eaaafa9611
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

In [ ]:
spark = SparkSession.builder.appName("expedia_analytics")\
.getOrCreate()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Part II: Data Readin & expands the data**

Read in main data

In [ ]:
main = spark.read.format("csv").option("header","true").load("drive/MyDrive/expedia/source/main.tsv",sep="\t")

In [ ]:
# preview of the data
main.show()

+--------------------+--------------------+-------------------+-------------+--------------------+---------+--------------+-------------------+-------------------+-----------+-----------+------------+----------+---------------+--------------------+--------------------+----+-----+--------------+
|             user_id|           search_id|   search_timestamp|point_of_sale|geo_location_country|is_mobile|destination_id|       checkin_date|      checkout_date|adult_count|child_count|infant_count|room_count|      sort_type|     applied_filters|         impressions|year|month|length_of_stay|
+--------------------+--------------------+-------------------+-------------+--------------------+---------+--------------+-------------------+-------------------+-----------+-----------+------------+----------+---------------+--------------------+--------------------+----+-----+--------------+
|001bbff3030c95740...|541933a931216d270...|2021-07-02 19:38:00|            2|                   1|        0|    

In [ ]:
# data name with data type
main.dtypes

[('user_id', 'string'),
 ('search_id', 'string'),
 ('search_timestamp', 'timestamp'),
 ('point_of_sale', 'string'),
 ('geo_location_country', 'string'),
 ('is_mobile', 'string'),
 ('destination_id', 'string'),
 ('checkin_date', 'timestamp'),
 ('checkout_date', 'timestamp'),
 ('adult_count', 'string'),
 ('child_count', 'string'),
 ('infant_count', 'string'),
 ('room_count', 'string'),
 ('sort_type', 'string'),
 ('applied_filters', 'string'),
 ('impressions', 'string'),
 ('year', 'int'),
 ('month', 'int'),
 ('length_of_stay', 'int')]

Read in amenities data

In [ ]:
amenities = spark.read.format("csv").option("header","true").load("drive/MyDrive/expedia/source/amenities.tsv",sep="\t")

In [ ]:
# preview of the data
amenities.show()

+-------+---------------+---------------+---+-------------------------+-------------+-----------+--------+---+-----------------+------+---------------+-------+-----------+-----------+-----------+------------+-----------+----+
|prop_id|AirConditioning|AirportTransfer|Bar|FreeAirportTransportation|FreeBreakfast|FreeParking|FreeWiFi|Gym|HighSpeedInternet|HotTub|LaundryFacility|Parking|PetsAllowed|PrivatePool|SpaServices|SwimmingPool|WasherDryer|WiFi|
+-------+---------------+---------------+---+-------------------------+-------------+-----------+--------+---+-----------------+------+---------------+-------+-----------+-----------+-----------+------------+-----------+----+
|      3|              0|              0|  0|                        0|            0|          1|       1|  0|                0|     0|              0|      0|          0|          0|          0|           0|          0|   0|
|      5|              0|              0|  0|                        0|            0|          0

Seperate impressions from the main data

In [ ]:
impr = main.select(["user_id","search_id","impressions"])

In [ ]:
impr_test = impr.withColumn("impressions",F.explode(F.split("impressions","\\|")))

In [ ]:
impr_test_seperate = impr_test.withColumn("rank",F.split(F.col("impressions"),",").getItem(0))\
  .withColumn("prop_id",F.split(F.col("impressions"),",").getItem(1))\
  .withColumn("is_travel_ad",F.split(F.col("impressions"),",").getItem(2))\
  .withColumn("review_rating",F.split(F.col("impressions"),",").getItem(3))\
  .withColumn("review_count",F.split(F.col("impressions"),",").getItem(4))\
  .withColumn("star_rating",F.split(F.col("impressions"),",").getItem(5))\
  .withColumn("is_free_cancellation",F.split(F.col("impressions"),",").getItem(6))\
  .withColumn("is_drr",F.split(F.col("impressions"),",").getItem(7))\
  .withColumn("price_bucket",F.split(F.col("impressions"),",").getItem(8))\
  .withColumn("num_clicks",F.split(F.col("impressions"),",").getItem(9))\
  .withColumn("is_trans",F.split(F.col("impressions"),",").getItem(10))\
  .drop("impressions")

In [ ]:
impr_test_seperate.show(n=10)

+--------------------+--------------------+----+-------+------------+-------------+------------+-----------+--------------------+------+------------+----------+--------+
|             user_id|           search_id|rank|prop_id|is_travel_ad|review_rating|review_count|star_rating|is_free_cancellation|is_drr|price_bucket|num_clicks|is_trans|
+--------------------+--------------------+----+-------+------------+-------------+------------+-----------+--------------------+------+------------+----------+--------+
|001bbff3030c95740...|541933a931216d270...|   1|5695234|           1|          4.0|       250.0|        2.0|                   1|     0|           3|         0|       0|
|001bbff3030c95740...|541933a931216d270...|   2|1734546|           1|          4.0|      1000.0|        2.0|                   1|     0|           4|         0|       0|
|001bbff3030c95740...|541933a931216d270...|   3|5675938|           0|          5.0|       700.0|        3.0|                   1|     0|           3| 

Merge impression data with main data

In [ ]:
merge_main = main.join(impr_test_seperate,["user_id","search_id"],how='inner').distinct()

In [ ]:
merge_main= merge_main.drop('impressions')

In [ ]:
# create extra column is_clicked, as we only care if a property is clicked, don't care the exact number of clicks
merge_main = merge_main.withColumn("is_clicked",F.when(F.col('num_clicks')=='0', 0).otherwise(1))

Merge with amenities data

In [ ]:
merge_all = merge_main.join(amenities,on=['prop_id'], how='left')

In [ ]:
merge_all.dtypes

[('prop_id', 'string'),
 ('user_id', 'string'),
 ('search_id', 'string'),
 ('search_timestamp', 'timestamp'),
 ('point_of_sale', 'double'),
 ('geo_location_country', 'string'),
 ('is_mobile', 'string'),
 ('destination_id', 'double'),
 ('checkin_date', 'timestamp'),
 ('checkout_date', 'timestamp'),
 ('adult_count', 'double'),
 ('child_count', 'double'),
 ('infant_count', 'double'),
 ('room_count', 'double'),
 ('sort_type', 'string'),
 ('applied_filters', 'string'),
 ('year', 'int'),
 ('month', 'int'),
 ('length_of_stay', 'int'),
 ('rank', 'string'),
 ('is_travel_ad', 'string'),
 ('review_rating', 'string'),
 ('review_count', 'string'),
 ('star_rating', 'string'),
 ('is_free_cancellation', 'string'),
 ('is_drr', 'string'),
 ('price_bucket', 'string'),
 ('num_clicks', 'string'),
 ('is_trans', 'string'),
 ('click_bool', 'int'),
 ('AirConditioning', 'string'),
 ('AirportTransfer', 'string'),
 ('Bar', 'string'),
 ('FreeAirportTransportation', 'string'),
 ('FreeBreakfast', 'string'),
 ('FreeP

# **Part III: statistical analysis**

General Numeric Summaries

In [ ]:
# count number of search history
main.count()

2577809

In [ ]:
# count distinct number of users
main.select(F.countDistinct("user_id")).show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 812638|
+-----------------------+



In [ ]:
# number of search based on customer's location
main.groupby("geo_location_country").count().sort(F.col("count").desc()).show()

+--------------------+-------+
|geo_location_country|  count|
+--------------------+-------+
|                   1|1784053|
|                   2| 171071|
|                   3|  85298|
|                   4|  60894|
|                   5|  57624|
|                   6|  55745|
|                   7|  53899|
|                   8|  53270|
|                   9|  25849|
|                  10|  20456|
|                  11|  17462|
|                  12|  17001|
|                  13|  12271|
|                  14|  10099|
|                  15|   8344|
|                  19|   6475|
|                  17|   6316|
|                  20|   6244|
|                  16|   6022|
|                  18|   5766|
+--------------------+-------+
only showing top 20 rows



In [ ]:
# number of search based on destination location
main.groupby("destination_id").count().sort(F.col("count").desc()).show() 

+--------------+-----+
|destination_id|count|
+--------------+-----+
|             2|52603|
|             3|38232|
|             1|35962|
|            71|26599|
|             4|21298|
|            10|19425|
|             6|17655|
|             8|16372|
|            18|16049|
|             5|14803|
|            12|14454|
|             9|12686|
|            11|12660|
|             7|12601|
|            15|12400|
|            33|11446|
|            13|11136|
|            31|10910|
|            32|10392|
|            16|10284|
+--------------+-----+
only showing top 20 rows



In [ ]:
# filter out positive clicked instances
clicked_records = merge_all.filter(F.col('is_clicked') == '1')

In [ ]:
# filter out positive transaction instances
booked_records = merge_all.filter(F.col('is_trans') == '1')

In [ ]:
# count positive clicked instances
clicked_records.count()

961595

In [ ]:
# count positive transanction instances
booked_records.count()

113380

In [ ]:
cols = ['rank','is_travel_ad','review_rating','review_count','star_rating',
        'is_free_cancellation','price_bucket','num_clicks','is_trans',
        'AirConditioning','AirportTransfer','Bar','FreeAirportTransportation',
        'FreeBreakfast','FreeParking','FreeWiFi','Gym','HighSpeedInternet',
        'HotTub','LaundryFacility','Parking','PetsAllowed','PrivatePool','SpaServices',
        'SwimmingPool','WasherDryer','WiFi']

In [ ]:
# 69% searches comes from country ID1, which could be united state
1784053/2577809

0.6920811433275312

In [ ]:
# filter out US data
main_nr1 = main.filter(F.col('geo_location_country') == '1')

In [ ]:
# count US users
main_nr1.select(F.countDistinct("user_id")).show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 531794|
+-----------------------+



In [ ]:
# 65% users located in us
531794/812638

0.6544045442127984

In [ ]:
# US destination country list
main_nr1.groupby("destination_id").count().sort(F.col("count").desc()).show() 

+--------------+-----+
|destination_id|count|
+--------------+-----+
|             2|48020|
|             3|33069|
|            71|24508|
|             4|18974|
|             1|16872|
|            10|16470|
|             6|15298|
|            18|14503|
|             5|14307|
|             8|12927|
|             9|12601|
|             7|11874|
|            11|11626|
|            12|11326|
|            15|11315|
|            33|10494|
|            13|10156|
|            28| 9459|
|            31| 9092|
|            16| 9077|
+--------------+-----+
only showing top 20 rows



# **Part4: Focus on us**

In [ ]:
# filter out US records
us = merge_main.filter(F.col('geo_location_country') == '1')

In [ ]:
# Random sample 60% of us data
us_sample= us.sample(0.6,123)

In [ ]:
# positive transaction instances from US sample
us_booked = us_sample.filter(F.col('is_trans')=='1')

In [ ]:
# negative transaction instances from US sample
us_notbooked = us_sample.filter(F.col('is_trans')=='0')

In [ ]:
# count positive transaction instances from US sample
us_booked.count()

55491

In [ ]:
# count negative transaction instances from US sample
us_notbooked.count()

75646830

In [ ]:
# look into frequency of num_clicks
us_sample.groupby("num_clicks").count().sort(F.col("count").desc()).show() 

+----------+--------+
|num_clicks|   count|
+----------+--------+
|         0|75289427|
|         1|  379475|
|         2|   27858|
|         3|    4163|
|         4|     949|
|         5|     283|
|         6|     107|
|         7|      25|
|         8|      18|
|         9|       8|
|        12|       3|
|        11|       2|
|        22|       1|
|        10|       1|
|        35|       1|
+----------+--------+



In [ ]:
# look into frequency of length of stay
us_sample.groupby("length_of_stay").count().sort(F.col("count").desc()).show() 


+--------------+--------+
|length_of_stay|   count|
+--------------+--------+
|             1|28855158|
|             2|14214528|
|             3|11106787|
|             4| 7745580|
|             5| 4827951|
|             7| 3679421|
|             6| 2522458|
|             8|  849486|
|             9|  537946|
|            10|  304275|
|            14|  250714|
|            11|  149711|
|            13|   94609|
|            12|   90879|
|            28|   89951|
|            15|   71671|
|            27|   51848|
|            21|   47308|
|            16|   42915|
|            17|   29585|
+--------------+--------+
only showing top 20 rows



In [ ]:
# look into frequency of star_rating
us_sample.groupby("star_rating").count().sort(F.col("count").desc()).show() 

+-----------+--------+
|star_rating|   count|
+-----------+--------+
|        3.0|24362414|
|       null|20427067|
|        4.0|19039365|
|        2.0| 6039869|
|        5.0| 5795333|
|        1.0|   37617|
|        0.0|     551|
|        6.0|     105|
+-----------+--------+



As the below shown, extreme low click through rate and transaction rate over all us related impressions.

In [ ]:
# filter out all us positive transaction impressions
us_booked_all = us.filter(F.col('is_trans')=='1')

In [ ]:
# filter out all us negative transaction impressions
us_notbooked_all = us.filter(F.col('is_trans')=='0')

In [ ]:
# count all us positive transaction impressions
us_booked_all.count()

92555

In [ ]:
# count all us negative transaction impressions
us_notbooked_all.count()

126071956

In [ ]:
# number of total us impressions
92555+126071956

126164511

In [ ]:
# Transaction rate over impressions
92555/(92555+126071956)

0.0007336056650669378

In [ ]:
# Transaction rate over search results
92555/1784053

0.0518790641309423

In [ ]:
# filter out all us negative click through impressions
us_notclick = us.filter(F.col('num_clicks')=='0')

In [ ]:
# count all us negative click through impressions
us_notclick.count()

125476375

In [ ]:
# total number of positive clicked instance
92555+126071956-125476375

688136

In [ ]:
# Click through rate over impressions
688136/(92555+126071956)

0.005454275489562988

In [ ]:
# Click through rate over search results
688136/1784053

0.38571499837729034